In [ ]:
import pandas as pd
import os
import zipfile

In [ ]:
# establish a dictionary for each tables purpose
grad_dict = {
    "table-1" : "Graduate activities by provider and sex",
    "table-2" : "Graduate activities by provider and age group",
    "table-3" : "Graduate activities by provider and disability",
    "table-4" : "UK domiciled graduate activities by provider and ethnicity",
    "table-5" : "Response rates by provider",
    "table-6" : "Graduate activities by provider",
    "table-8" : "Graduate reflectionsd by provider"
}

In [ ]:
files_dir = 'extracted-tables'

def combine_tables(tables_to_extract):
    for table in tables_to_extract:
        all_data = []
        for root, _, files in os.walk(files_dir):
            for file in files:
                if table in file:
                    file_path = os.path.join(root, file)
                    table_name = file[:7]  # Get the table name
                    table_year = file[-11:-4]  # Get the year of the data
                    
                    df = pd.read_csv(file_path, low_memory=False)
                    header_row_index = None
                    for index, row in df.head(20).iterrows():
                        if 'UKPRN' in row.values:
                            header_row_index = index
                            break
                    if header_row_index is not None:
                        df.columns = df.iloc[header_row_index]
                        df = df[header_row_index + 1:].reset_index(drop=True)
                        
                    df['tableName'] = grad_dict.get(table_name, 'Unknown Table Name')  # Get full table name
                    df['tableYear'] = table_year
                    df['sourceTable'] = table_name

                    # Combine into a list
                    all_data.append(df)

        if all_data:  # Ensure there is data to concatenate
            fdf = pd.concat(all_data, ignore_index=True)  # Combine all dataframes in the list
            output_file = f"{table}-go-data.csv"  # Define the output file name
            fdf.to_csv(output_file, index=False)  # Export as a CSV
            print(f"Finished exporting -> {output_file}")
        else:
            print(f"No data found for table: {table}")

combine_tables(tables_to_extract=['table-1', 'table-2', 'table-3', 'table-4', 'table-6', 'table-8'])